#### Data preperation for finetuning GPT2 with AVA-captions dataset

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import pipeline

Used code from: https://rowlando13.medium.com/everything-gpt-2-5-fine-tuning-885aec508c4

In [ ]:
model_path = "../CLIP-GPT2/models/gpt2-large"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
tokenizer.save_pretrained(model_path)

model = GPT2LMHeadModel.from_pretrained("gpt2-large")
model.save_pretrained(model_path)

In [ ]:
## DATA PREPERATION
def file_to_list(text_loc, tokenizer, max_len):
    text_blocks = []
    
    f = open(text_loc, encoding="utf-8")
    text = f.read()
    
    tokens = tokenizer.encode(text)
    
    while len(tokens) > 0:
        holder = []
        if len(tokens) > max_len:
            holder = tokens[0:max_len]
            del tokens[0:max_len]
            if holder[-1] != 50256:
                holder.append(50256)
        
        else:
            holder = tokens
            tokens = []
            if holder[-1] != 50256:
                holder.append(50256)
                
        text_blocks.append(tokenizer.decode(holder))
    
    return text_blocks

def text_to_pieces(text_loc, tokenizer, max_len=1024):
    text_blocks = file_to_list(text_loc, tokenizer, max_len)    
    return text_blocks

text_loc = "../datasets/AVA/AVA-captions_clean_full_text.txt"
max_len = 1024

data = text_to_pieces(text_loc, tokenizer, max_len)

In [ ]:
train, test = train_test_split(data, test_size=0.1, random_state=42)

In [ ]:
train_loc = "../CLIP-GPT2/data/large/train.txt"
test_loc = "../CLIP-GPT2/data/large/test.txt"

In [ ]:
train_text = ""
train_text = train_text.join(train)

test_text = ""
test_text = test_text.join(test)

with open(train_loc, "x", encoding="utf-8") as f:
    f.write(train_text)

with open(test_loc, "x", encoding="utf-8") as f:
    f.write(test_text)